In [1]:
import sys
sys.path.append("../src")

# If you do some changes in file. jupiter notebook will update saved files 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
#name file of doccano and explanation maps 
file='rationale'

In [6]:
import captum.attr as a

from dataset import OurDataset
from explain import STS_ExplainWrapper, ExplanationExecuter_STS, compare_multiple_explanation_methods
from check_explanations import Check_docano_XAI 
from captum._utils.models.linear_model import SkLearnLasso
from captum.attr._core.lime import get_exp_kernel_similarity_function

In [7]:
dataset = OurDataset(csv_dirpath="./data")

In [8]:
model = STS_ExplainWrapper.setup_transformer("../models/GTR-T5-FT",'encoder.embed_tokens',interpretable_embeddings=True)

c:\Users\Dell\anaconda3\envs\env_autoxai_project\lib\site-packages\captum\attr\_models\base.py:191: UserWarning: In order to make embedding layers more interpretable they will be replaced with an interpretable embedding layer which wraps the original embedding layer and takes word embedding vectors as inputs of the forward function. This allows us to generate baselines for word embeddings and compute attributions for each embedding dimension. The original embedding layer must be set back by calling `remove_interpretable_embedding_layer` function after model interpretation is finished. 
  warnings.warn(


In [6]:
#get embedding layer: setup_t5_transformer.get_embedding_layer()

In [9]:
exp_eucl_distance = get_exp_kernel_similarity_function('euclidean', kernel_width=450) #related to Lime
# TODO more LayerXAI methods need to be tested",
# a.LayerGradientXActivation(model, model.get_embedding_layer())
methods = [
    a.Lime(model,similarity_func=exp_eucl_distance,interpretable_model=SkLearnLasso(alpha=1e-19)),
    a.Saliency(model),
    a.Occlusion(model),
    a.InputXGradient(model),
    a.GuidedBackprop(model),
    a.Deconvolution(model),
    a.GradientShap(model,multiply_by_inputs=False),
    a.KernelShap(model),
    a.IntegratedGradients(model),
    a.FeatureAblation(model),
    a.DeepLift(model),
    a.ShapleyValueSampling(model),
    # a.GuidedGradCam(model,layer=model.model.hf_transformer.encoder.embed_tokens.embedding),
    a.LRP(model)
]
explain_wrapper_kwargs = [
    {"parameters":{"n_samples":450,'perturbations_per_eval':1}, "token_groups_for_feature_mask": True },       ### Lime
    {"parameters":{"abs": True}},                                                 ### Saliency
    { "parameters":{"sliding_window_shapes":(1,1024),'strides':(1,1024)},"compute_baseline": True},     ### Occlusion
    {},                                                                            ### Input X Gradient
    {},                                                                            ### Guided Backprop "parameters":{"Layer":True}
    {},                                                                            ### Deconvolution
    {"compute_baseline": True, "parameters":{'stdevs':1.0,'n_samples':25}},   ### GradientShap
    {"token_groups_for_feature_mask": True,"parameters":{'n_samples':1000}},       ### KernelShap
    { "compute_baseline": True, "token_groups_for_feature_mask": True },
    {"token_groups_for_feature_mask": True,'parameters':{'n_samples':5}},
]
method_names = [
    "Lime",
    "Saliency",
    'Occlusion',
    "Input X Gradient",
    "Guided Backprop",
    "Deconvolution",
    'Gradient Shap',
    'Kernel Shap'
    "Integrated Gradients",
    "Feature Ablation",
    "DeepLift",
    "Shapley Value Sampling",
    "LRP"
    # "GuidedGradCam"
]

In [10]:
explain_wrappers = []
for method, kwargs in zip(methods, explain_wrapper_kwargs):
    explain_wrappers.append(
        ExplanationExecuter_STS(method,**kwargs)
    )


In [11]:
check=Check_docano_XAI(rationale_dataset_path=f'../data/annotations/{file}.json',xai_dataset=dataset,tuple_indexes=dataset.fact_check_post_mapping) #data=doccano importance_map=xai
indexes_doc, indexes_xai, doc_data=check.get_matched_doccano()

In [ ]:
for i in indexes_xai[0:1]:#visualization
    claim, post = dataset[i]
    compare_multiple_explanation_methods(explain_wrappers, post, claim, additional_attribution_kwargs= {}, method_names=method_names) 